In [1]:
%run base.ipynb

# EPIC 1 - User Story DF-24
## Erstellung eines Notebooks und aufrufen der Daten per Dataland-API

Als QA-Lab-Team möchten wir ein Notebook erstellen, mit dem wir eine Anfrage an die Dataland-API senden, sodass wir anhand der Berichts-ID den Unternehmensbericht erhalten.

Beschreibung:<br>
Nutzen von Dataland Endpoint /data/nuclear-and-gas/companies/{companyId} im Nuclear & Gas Controller. Die Funktionalität wird außerhalb des Notebooks implementiert.

Akzeptanzkriterien:<br>
    - Notebook ist auf jedem Laptop ausführbar<br>
    - Anfrage an Dataland gibt Daten zurück

## 1.Schritt: Dataset mit Hilfe der Company_ID über die Dataland_API aufrufen
Mithilfe der Company_ID wird die Dataland_API aufgerufen. Nachdem Eintragen aus welcher Periode das Dataset sein soll, ist es möglich den Wert 1 zu erlangen.

In [ ]:
import dataland_qa_lab.dataland.get_data as qa

company_id = "4423c691-0436-423f-abcb-0a08127ee848"
year = "2024"

# Laden aller Datasets aus jeder Periode
all_datasets = qa._get_all_company_datasets_(company_id=company_id)

# Data_ID der gewünschten Periode des Datensets erhalten
data_id = qa._get_data_id_by_year_(company_id=company_id, year=year)
print("Data_ID = " + data_id)

# Laden eines bestimmten Datensets einer bestimmten Periode
dataset_by_year = qa._get_dataset_by_year_(company_id=company_id, year=year)

# Rückgabe des ersten Wertes in dem bestimmten Datenset
value1 = qa._get_value1_by_year_(company_id=company_id, year=year)
print("Value 1 = " + value1)

# Erhalten der Datenreferenz in Bytes, wenn eine Datenquelle hinterlegt ist
datasource_reference_bytes = qa._get_datasource_reference_bytes_(company_id=company_id, year=year)

Data_ID = d67db903-575d-48c3-99c3-b7f3f8ea174f
Value 1 = Yes


## Laden aller 6 JA/ NEIN Fragen
Hier werden die Daten von Dataland geladen und alle 6 Werte gesondert gespeichert.

from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

api = dataland_client.eu_taxonomy_nuclear_and_gas_api
dataset = api.get_all_company_nuclear_and_gas_data(company_id=company_id)

# Eintragen aus welcher Periode man das Dataset haben will -> In diesem Fall 2024
data_id = "test"
for t in range(len(dataset)):
    if (dataset[t].meta_info.reporting_period == year):
        data_id = dataset[t].meta_info.data_id
        break

data = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_company_associated_nuclear_and_gas_data(data_id=data_id)

wert1 = data.data.general.general.nuclear_energy_related_activities_section426
wert2 = data.data.general.general.nuclear_energy_related_activities_section427
wert3 = data.data.general.general.nuclear_energy_related_activities_section428
wert4 = data.data.general.general.fossil_gas_related_activities_section429
wert5 = data.data.general.general.fossil_gas_related_activities_section430
wert6 = data.data.general.general.fossil_gas_related_activities_section431
print(data)
print(wert1)
print(wert2)
print(wert3)
print(wert4)
print(wert5)
print(wert6)

## Daten aus dem Dokument extrahieren und in Text umwandeln - DEMO

import io

import pypdf

full_document_byte_stream = io.BytesIO(document_bytes)
full_pdf = pypdf.PdfReader(full_document_byte_stream)

partial_document_byte_stream = io.BytesIO()
partial_pdf = pypdf.PdfWriter()

partial_pdf.add_page(full_pdf.get_page(int(wert1.data_source.page) - 1))  # Correct for 0 offset
partial_pdf.write(partial_document_byte_stream)
partial_document_byte_stream.seek(0)
None

from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from azure.core.credentials import AzureKeyCredential

docintel_cred = AzureKeyCredential(conf.azure_docintel_api_key)
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=conf.azure_docintel_endpoint, credential=docintel_cred
)

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout",
    analyze_request=partial_document_byte_stream,
    content_type="application/octet-stream",
    output_content_format=ContentFormat.MARKDOWN,
)
result: AnalyzeResult = poller.result()

#### Das Ergebnis wird als Markdown direkt im Notebook wiedergegeben

from IPython.display import Markdown, display

display(Markdown(result.content))